In [2]:
from context import *

# Stablizer Formalism (`stabilizer`)

## General Idea: State-Map Duality

Every stabilizer state $\rho$ is dual to a Clifford unitary $U$, such that the state can be generated from the zero state $|00\cdots0\rangle$ as
$$\rho = U|00\cdots0\rangle\langle 00\cdots0|U^\dagger$$.
Both $\rho$ and $U$ describes a stabilizer code:
* $\rho$ is a projection operator that specifies the code subspace of the stabilizer code.
* $U$ is the encoding Clifford unitary that encodes the logical + syndrome qubits to the physical qubits in the stabilizer code.

The package `stabilizer` (based on `paulialg`) provides related functions to represent stabilizer states and Clifford maps. There are two classes defined in this package.

* `stabilizer.CliffordMap`. Since the Clifford unitary $U$ maps Pauli operators to Pauli operators, it is sufficient to specify a Clifford unitary by how each single-qubit Pauli operator transforms under the unitary. Such transformation rules are stored in a table called the Clifford map.

* `stabilizer.StabilizerState`. The stabilizer state is specified by a set of stabilizers and the corresponding destabilizers. Using the binary representation of Pauli operators, they can be stored in a table, called the stabilizer tableau. 

Since both classes need to store a table of Pauli operators, they are both realized as subclasses of `paulialg.PauliList`.

## Basic Usage

---

### Constructors

#### Construct Clifford Maps

`identity_map(N)` constructs an identity Clifford map on $N$ qubits.

In [2]:
stabilizer.identity_map(4)

CliffordMap(
  X0-> +XIII
  Z0-> +ZIII
  X1-> +IXII
  Z1-> +IZII
  X2-> +IIXI
  Z2-> +IIZI
  X3-> +IIIX
  Z3-> +IIIZ)

`random_pauli_map(N)` samples a random Clifford map made of random single-qubit Clifford gates on $N$ qubits, i.e. $U=\prod_i U_i\in\mathrm{Cl}(2)^N$. Each realization specifies a random local Pauli basis.

In [3]:
stabilizer.random_pauli_map(4)

CliffordMap(
  X0-> -YIII
  Z0-> +XIII
  X1-> -IXII
  Z1-> +IZII
  X2-> -IIYI
  Z2-> -IIZI
  X3-> +IIIX
  Z3-> -IIIZ)

`random_clifford_map(N)` samples a globally random Clifford map on $N$ qubits, i.e. $U\in\mathrm{Cl}(2^N)$. Each realization specifies a random global stabilizer basis.

In [5]:
stabilizer.random_clifford_map(4)

CliffordMap(
  X0-> +XYYI
  Z0-> -IZIX
  X1-> +ZZII
  Z1-> +IYIY
  X2-> -XXZZ
  Z2-> -ZIZI
  X3-> +YXYZ
  Z3-> +YYXY)

`clifford_rotation_map(N)` constructs a Clifford map based for a Clifford rotation given its generator.

In [6]:
stabilizer.clifford_rotation_map('-XXYZ')

CliffordMap(
  X0-> +XIII
  Z0-> +YXYZ
  X1-> +IXII
  Z1-> +XYYZ
  X2-> +XXZZ
  Z2-> -XXXZ
  X3-> -XXYY
  Z3-> +IIIZ)

#### Construct Stabilizer States

`maximally_mixed_state(N)` constructs a $N$-qubit maximally mixed state (by setting the density matrix to full rank).
$$\rho=2^{-N}\mathbb{1}.$$

In [9]:
stabilizer.maximally_mixed_state(4)

StabilizerState()

`zero_state(N)` constructs a $N$-qubit all-zero state 
$$\rho=|0\cdots0\rangle\langle 0\cdots0|=\prod_{i}\frac{1+Z_i}{2}.$$

In [10]:
stabilizer.zero_state(4)

StabilizerState(
   +ZIII
   +IZII
   +IIZI
   +IIIZ)

`one_state(N)` constructs a $N$-qubit all-one state 
$$\rho=|1\cdots1\rangle\langle 1\cdots1|=\prod_{i}\frac{1-Z_i}{2}.$$

In [11]:
stabilizer.one_state(4)

StabilizerState(
   -ZIII
   -IZII
   -IIZI
   -IIIZ)

`ghz_state(N)` constructs a $N$-qubit GHZ state
$$\rho = |\Psi\rangle\langle\Psi|, \qquad \text{with }|\Psi\rangle=\frac{1}{\sqrt{2}}(|0\cdots0\rangle+|1\cdots1\rangle).$$

In [12]:
stabilizer.ghz_state(4)

StabilizerState(
   +XXXX
   +IIZZ
   +IZZI
   +ZZII)

`random_pauli_map(N)` samples a $N$ qubit random Pauli state.
$$\rho=U|0\cdots0\rangle\langle 0\cdots0|U^\dagger,\qquad\text{with }U\in \mathrm{Cl}(2)^N.$$

In [14]:
stabilizer.random_pauli_map(4)

CliffordMap(
  X0-> +XIII
  Z0-> -ZIII
  X1-> -IXII
  Z1-> +IZII
  X2-> -IIYI
  Z2-> +IIXI
  X3-> -IIIZ
  Z3-> +IIIX)

`random_clifford_map(N)` samples a $N$ qubit random Clifford (random stabilizer) state.
$$\rho=U|0\cdots0\rangle\langle 0\cdots0|U^\dagger,\qquad\text{with }U\in \mathrm{Cl}(2^N).$$

In [15]:
stabilizer.random_clifford_state(4)

StabilizerState(
   +ZZZY
   -XZIZ
   +YYYZ
   +IYZX)

`stabilizer_state(...)` is a universal constructor of stabilizer state by specifying all stabilizers.

In [22]:
stabilizer.stabilizer_state('XXY','ZZI','IZZ')

StabilizerState(
   +IZZ
   +ZZI
   +XXY)

A hack to inspect the full stabilizer tableau is by converting `StabilizerState` to `PauliList` by

In [23]:
stabilizer.stabilizer_state('XXY','ZZI','IZZ')[:]

 +IZZ
 +ZZI
 +XXY
 +ZIX
 +ZXX
 +ZII

<div class="alert alert-block alert-danger">
User need to <b>ensure</b> that stabilizers commute with each other, otherwise an error will be raised.
</div>

In [24]:
stabilizer.stabilizer_state('XXY','-YYI','IZZ')

ValueError: stabilizers must all commute with each other.

#### State-Map Conversion

Stabilizer states and Clifford maps can be mapped to each other.

In [26]:
rho = stabilizer.stabilizer_state('XXX','ZZI','IZZ')
print("quantum state: \n", rho)

quantum state: 
 StabilizerState(
   +IZZ
   +ZZI
   +XXX)


In [27]:
rho.to_map()

CliffordMap(
  X0-> +IIX
  Z0-> +IZZ
  X1-> +IXX
  Z1-> +ZZI
  X2-> +ZII
  Z2-> +XXX)

And Clifford map can be converted back to the stabilizer state.

In [28]:
rho.to_map().to_state()

StabilizerState(
   +IZZ
   +ZZI
   +XXX)

* `.to_map()` and `.to_state()` will make **new copies** of Pauli string data in the memory.

<div class="alert alert-block alert-danger">
The information about the rank of the density matrix is lost in the Clifford map, so the back conversion will result in a zero rank stabilizer state.
</div>

---

### Clifford Map Methods

#### - Map Embedding

`.embed(small_map, mask)` provides the method to embed a smaller Clifford map on a subset of qubits to the current Clifford map. This is a **in-place** operation. The Clifford map object that provide this method will get modified under the embedding.

**Parameters:**
* `small_map` is a `CliffordMap` object supported on a subset of qubits.
* `mask` is a boolean array specifying the subset of qubits.

In [29]:
cmap = stabilizer.identity_map(6)
cmap

CliffordMap(
  X0-> +XIIIII
  Z0-> +ZIIIII
  X1-> +IXIIII
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +IIIXII
  Z3-> +IIIZII
  X4-> +IIIIXI
  Z4-> +IIIIZI
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

In [30]:
cmap.embed(stabilizer.random_clifford_map(3), numpy.array([True,False,False,True,True,False]))

CliffordMap(
  X0-> +XIIIXI
  Z0-> +YIIXXI
  X1-> +IXIIII
  Z1-> +IZIIII
  X2-> +IIXIII
  Z2-> +IIZIII
  X3-> +IIIIXI
  Z3-> -ZIIXZI
  X4-> +IIIXII
  Z4-> +XIIYII
  X5-> +IIIIIX
  Z5-> +IIIIIZ)

#### - Map Composition

`.compose(other)` returns the composition of the current Clifford map with another Clifford map. This will return a new Clifford map without modifying either of the input maps. The Clifford map object which initiates this method will be the preceeding map in the composition. 

**Parameters:**
* `other` - another `CliffordMap`.

In [34]:
cmap.compose(stabilizer.random_clifford_map(6))

CliffordMap(
  X0-> +XYYXXZ
  Z0-> -XYXYIY
  X1-> +XIXXYZ
  Z1-> +XZIIZZ
  X2-> -ZZXIIY
  Z2-> -XYXZXZ
  X3-> +XZZYXY
  Z3-> -ZYZYZI
  X4-> +ZZZYXZ
  Z4-> +IIZIZZ
  X5-> -XXXXZY
  Z5-> +YZXXIX)

#### - Map Inversion

`.inverse()` returns the inverse of the current Clifford map. This will return a new Clifford map withoutt modifying the original map. The inverse map is such that its composition with the original map must be identity

In [37]:
cmap = stabilizer.clifford_rotation_map('Y')
cmap

CliffordMap(
  X0-> -Z
  Z0-> +X)

In [38]:
cmap.inverse()

CliffordMap(
  X0-> +Z
  Z0-> -X)

Test on random maps.

In [44]:
cmap = stabilizer.random_clifford_map(20)
cmap.inverse().compose(cmap)

CliffordMap(
  X0 -> +XIIIIIIIIIIIIIIIIIII
  Z0 -> +ZIIIIIIIIIIIIIIIIIII
  X1 -> +IXIIIIIIIIIIIIIIIIII
  Z1 -> +IZIIIIIIIIIIIIIIIIII
  X2 -> +IIXIIIIIIIIIIIIIIIII
  Z2 -> +IIZIIIIIIIIIIIIIIIII
  X3 -> +IIIXIIIIIIIIIIIIIIII
  Z3 -> +IIIZIIIIIIIIIIIIIIII
  X4 -> +IIIIXIIIIIIIIIIIIIII
  Z4 -> +IIIIZIIIIIIIIIIIIIII
     ...
  X15-> +IIIIIIIIIIIIIIIXIIII
  Z15-> +IIIIIIIIIIIIIIIZIIII
  X16-> +IIIIIIIIIIIIIIIIXIII
  Z16-> +IIIIIIIIIIIIIIIIZIII
  X17-> +IIIIIIIIIIIIIIIIIXII
  Z17-> +IIIIIIIIIIIIIIIIIZII
  X18-> +IIIIIIIIIIIIIIIIIIXI
  Z18-> +IIIIIIIIIIIIIIIIIIZI
  X19-> +IIIIIIIIIIIIIIIIIIIX
  Z19-> +IIIIIIIIIIIIIIIIIIIZ)

In [45]:
cmap.compose(cmap.inverse()) 

CliffordMap(
  X0 -> +XIIIIIIIIIIIIIIIIIII
  Z0 -> +ZIIIIIIIIIIIIIIIIIII
  X1 -> +IXIIIIIIIIIIIIIIIIII
  Z1 -> +IZIIIIIIIIIIIIIIIIII
  X2 -> +IIXIIIIIIIIIIIIIIIII
  Z2 -> +IIZIIIIIIIIIIIIIIIII
  X3 -> +IIIXIIIIIIIIIIIIIIII
  Z3 -> +IIIZIIIIIIIIIIIIIIII
  X4 -> +IIIIXIIIIIIIIIIIIIII
  Z4 -> +IIIIZIIIIIIIIIIIIIII
     ...
  X15-> +IIIIIIIIIIIIIIIXIIII
  Z15-> +IIIIIIIIIIIIIIIZIIII
  X16-> +IIIIIIIIIIIIIIIIXIII
  Z16-> +IIIIIIIIIIIIIIIIZIII
  X17-> +IIIIIIIIIIIIIIIIIXII
  Z17-> +IIIIIIIIIIIIIIIIIZII
  X18-> +IIIIIIIIIIIIIIIIIIXI
  Z18-> +IIIIIIIIIIIIIIIIIIZI
  X19-> +IIIIIIIIIIIIIIIIIIIX
  Z19-> +IIIIIIIIIIIIIIIIIIIZ)

---

### Stabilizer State Methods

#### - Measurement

`.measure(obs)` measure the stabilizer state on a set of commuting observables.

**Parameters:**
* `obs` - Observables to measure. The following types are supported:
    * `PauliList` - a list of Pauli operators (user must ensure that operators in the list are commuting, otherwise they can not measured simutaneously).
    * `StabilizerState` - stabilizers of a stabilizer state is always commuting, which can be treated as commuting observables for measurement.

**Returns:**
* `out` - measuremnt outcome, can only be $0$, $\pm1$ for independent Pauli observables on stabilizer state.
* `log2prob` - the log2 of the probability of realizing this particular outcome.

## A little hack into the design

---

### StabilizerState

The basic classes in stabilizer are: 1. StabilizerState and 2. CliffordMap, which are linked through state-map duality

#### The Representation of Density Matrix

A $[N,r]$ stabilizer state is describe by the **density matrix** of the following form:
$$\rho = \frac{1}{2^r}\prod_{k=1}^{N-r}\frac{1+(-)^{b_k}S_k}{2}.$$
* Each stabilizer $S_k$ is a (non-trivial) Pauli operator defined on totally $N$ qubits. The stabilizers commute with each other $[S_k,S_{k'}]=0$. They generate an Abelian subgroup $\mathcal{S}=\{\prod_{k=1}^{N-r} S_k^{a_k}|a_k=0,1\}$ of the $N$-qubit Pauli group, called the *stabilizer group*.
* Each sign indicator $b_k=0,1$ is a binary variable specifying the eigen space of the stabilizer.
* There are totally $N-r$ stabilizers for a $[N,r]$ stablizer code (of code rate: $r/N$). The simultaneous eigenspace of all stabilizers constitutes the *code subspace*. 
* The code subspace is $2^r$ dimensional (which is also the rank of the density matrix $\rho$). The stabilizer state $\rho$ is always defined to be the maximally mixed state in the code subspace, such that $\rho$ is also the **projection operator** that projects any state into the code subspace.

#### Binary Representation of Pauli Operators

Any Pauli operator can be specified by two one-hot (binary) vectors $x$ and $z$ ($x_i,z_i=0,1$ for $i=1,\cdots,N$):
$$\sigma_{(x,z)}=\mathrm{i}^{x\cdot z}\prod_{i=1}^{N}X_i^{x_i}\prod_{i=1}^{N}Z_i^{z_i}.$$
* The binary vector $x$ (or $z$) specifies the qubits where the $X$ (or $Z$) operator acts ($Y$ operator acts at where $X$ and $Z$ act simultaneously).
* **Multiplication** of two Pauli operators
$$\sigma_{(x,z)}\sigma_{(x',z')}=\mathrm{i}^{p(x,z;x',z')}\sigma_{(x+x',z+z')\%2},$$
where the power $p$ of $\mathrm{i}$ in the prefactor is given by
$$p(x,z;x',z')=\sum_{i=1}^{N}\left(z_ix'_i-x_iz'_i + 2(z_i+z'_i)\left\lfloor\frac{x_i+x'_i}{2}\right\rfloor+2(x_i+x'_i)\left\lfloor\frac{z_i+z'_i}{2}\right\rfloor\right)\mod 4.$$
* **Commutation relation**: two Pauli operator either commute to anticommute.
$$\sigma_{(x,z)}\sigma_{(x',z')}=(-)^{c(x,z;x',z')}\sigma_{(x',z')}\sigma_{(x,z)},$$
where the *anticommutation indicator* $c$ has a simpler form
$$c(x,z;x',z')=\frac{p(x,z;x',z')-p(x',z';x,z)}{2}=\sum_{i=1}^{N}\left(z_ix'_i-x_iz'_i\right)\mod 2.$$

The binary vectors $x$ and $z$ can be interweaved into a $2N$-component vector $g=(x_0,z_0,x_1,z_1,\cdots)$, which forms the binary representation of a Pauli operator $\sigma_g$.

#### Stabilizer Tableau

Each stabilizer state is internally stored in the form of a **stabilizer tableau** $S$, together with the **sign indicator** $b$. For a $[N,r]$ stabilizer state, its stabilizer tableau is a $2N\times 2N$ matrix of the following structure

<img src="fig_tableau.png" alt="stabilizer tableau." style="width: 320px;"/>

* Each row is a binary representation $(x,z)$ of a Pauli oparator $\sigma_{(x,z)}$.
* Totally $2N$ Pauli operators grouped into $N$ stabilizers and $N$ destabilizers, such that for $i,j=0,\cdots,2N-1$:
$$\sigma_{g_{i}}\sigma_{g_{j}}=(-)^{\delta_{i+N,j}-\delta_{j+N,i}}\sigma_{g_{j}}\sigma_{g_{i}},$$
i.e. the $i$th stabilizer only anticommute with the $i$th destabilizer and they commute with all the other operators in the tableau.
* The rows $r:N$ correspond to the $N-r$ active stabilizers $S_k$, which stabilize the code subspace (impleted as projection operators). The rows $0:r$ corresponds to the $r$ standby (inactive) stabilizers that does not realy stabilizer the code subspace (but they will act as logical operators in the code subspace).
* The rows $N+r:2N$ correspond to the $N-r$ active destabilizers that anticommute with the active stabilizers. The rows $N:N+r$ correspond to the $r$ standby destabilizers taht anticommute with the standby stabilizers.

Although the stabilizer state is only specified by the active stabilizers, the other operators in the stabilizer tableau are still important in order to complete the operator basis. Such that the tableau can specify an unitary operator in the Clifford group that generate the state. The algorithm must mantain the algebraic structure betwen all stabilizers and destabilizers while updating the tableau.

Example: stabilizer tableau is given by `StabilizerState.gs`

In [50]:
rho = stabilizer.random_clifford_state(4)
print('state is: \n',rho)

state is: 
 StabilizerState(
   -YIYX
   +ZXYY
   -XIZI
   -ZZYZ)


In [51]:
rho.gs

array([[1, 1, 0, 0, 1, 1, 1, 0],
       [0, 1, 1, 0, 1, 1, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 1, 0, 1],
       [1, 1, 1, 0, 1, 1, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 1],
       [1, 1, 1, 0, 0, 1, 1, 1],
       [0, 1, 0, 0, 1, 1, 1, 1]])

The sign of (de)stabilziers can be read through `StabilizerState.ps`, which returns the power of $i$

In [55]:
rho.ps

array([2, 0, 2, 2, 0, 0, 0, 2])

In [5]:
stabilizer.random_pauli_map(3)

CliffordMap(
  X0-> +ZII
  Z0-> +XII
  X1-> -IZI
  Z1-> +IYI
  X2-> +IIY
  Z2-> +IIX)